# Saving to file

Once a turbulence box has been generated, it can be converted to either a HAWC2 turbulence binary file (.bin) or a full-field TurbSim binary file (.bts).

Here is a quick example that generates turbulence, saves it to file and then reloads it.

**Note**: These read/write functions are only applicable to 3D turbulence boxes generated on a grid with a specific column naming convention/order. Please either use `gen_spat_grid` combined with `gen_turb` or be very sure that your column ordering/naming in your dataframe is consistent with PyConTurb's ordering.

In [ ]:
# import needed functions
from pyconturb import gen_spat_grid, gen_turb
from pyconturb.io import df_to_h2turb, h2turb_to_df, df_to_bts, bts_to_df
import matplotlib.pyplot as plt

In [ ]:
# create the turbulence box
nt, dt = 100, 0.2
spat_df = gen_spat_grid(0, [50, 70])  # two points
turb_df = gen_turb(spat_df, T=nt*dt, dt=dt, u_ref=12)

### HAWC2 turbulence format

Because the HAWC2 binary format does not include all necessary information to save/reconstruct the turbulence box (e.g., the grid points), we must also pass `spat_df` into the functions. There are 3 output files that have endings "u.bin", "v.bin" and "u.bin". The `prefix` keyword argument allows you to prepend a string in from of the endings.

In [ ]:
# save in HAWC2 format (u.bin, v.bin, w.bin)
path, prefix = '.', 'h2turb_'  # path is directory, prefix is prepended to "u.bin", "v.bin", "w.bin"
df_to_h2turb(turb_df, spat_df, path, prefix=prefix)  # save file

# reload file
h2_df = h2turb_to_df(spat_df, path, prefix=prefix, nt=nt, dt=dt)

# compare the original and reloaded turbulence files
plt.plot(turb_df.u_p1, label='Original')
plt.plot(h2_df.u_p1, label='Reloaded')
plt.legend();

### TurbSim turbulence format

Unlike the HAWC2 format, the full-field TurbSim binary format stores all three components as well as all information required to reconstruct the turbulence box. However, we are still required to pass in `spat_df` on saving, because we need to encode the spatial information in `spat_df` into the output binary file.

Because TurbSim requires specifying a hub height and corresponding wind speed, the write function allows you to specify those values directly. If no coordinates are given, the center of the grid is used.

In [ ]:
# save in TurbSim format (path.bts)
path = 'ts.bts'  # path is name of file (with or without ending)
df_to_bts(turb_df, spat_df, path, uzhub=None)  # assume hub height is in center of box

# reload file
ts_df = bts_to_df(path)

# compare the original and reloaded turbulence files
plt.plot(turb_df.u_p1, label='Original')
plt.plot(ts_df.u_p1, label='Reloaded')
plt.legend();